# Hill Cypher Encryption
---

In this example, we'll be using a linear transformation (and its inverse) to encode and decode a string of lowercase characters.

In [22]:
# importing numpy for linear algebra operations, scipy for sparse matrices
import numpy as np
from scipy.sparse import coo_matrix

## Integer representation of the message string
---
Our goal here will be to take the string "ibotta" and encrypt it via a linear transformation. To do this, we'll first map each character in the lowercase string to the integer range [0, 25] (where a = 0, b = 1, ..., z = 25). Here is the result of that mapping:

```
i -> 8
b -> 1
o -> 14
t -> 19
t -> 19
a -> 0
```

We'll use these values to populate an integer vector representation of the string. I'm going to initialize a (6,1) numpy matrix to do this. Note the transpose ('T') used to convert the default interpretation of a *row* vector/matrix to the *column* format we want.

In [20]:
messageIntVec = np.matrix([8, 1, 14, 19, 19, 0]).T
print(messageIntVec)

[[ 8]
 [ 1]
 [14]
 [19]
 [19]
 [ 0]]


## Forward transformation (encryption) - altering *one* letter
---
There are more general ways of constructing Hill cyphers/transformations than what we'll explore today (and you can read more at https://en.wikipedia.org/wiki/Hill_cipher). For our illustration, though, we'll use a simple composition that'll allow us to easily determine a forward (encryption) and inverse (decryption) transformation.

Let's say we want to use the secret key 'mykey' to encrypt the information. Here's one way we can do that.

First, as we did with the 'ibotta' string, let's transform the 'mykey' string into an integer array representation.

```
m -> 12
y -> 24
k -> 10
e -> 4
y -> 24
```

I'm actually going to keep these values as a *row* vector because of how they'll be subsequently used.

In [24]:
keyIntArray = np.array([12, 24, 10, 4, 24])

We'll use these values to start transforming the letters of our 'ibotta' message, one by one. Below is the construction (using sparse matrix logic as a 'helper') of a matrix operator that only transforms the *last* element in our 'ibotta' integer vector.

In [34]:
row    = np.array([5, 5, 5, 5, 5])
col    = np.array([0, 1, 2, 3, 4])
values = keyIntArray

firstTransformation = coo_matrix((values, (row, col)), shape = (6, 6)).toarray() + np.identity(6)
print(firstTransformation)

[[ 1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.]
 [12. 24. 10.  4. 24.  1.]]


From what we know about matrix-vector multiplication, the action of this matrix on our message integer vector results in the first five elements being left alone, with the sixth (representing the letter 'a' in 'ibotta') being replaced by its integer representation (0 in the case of 'a') *plus a linear combination of the other integers in the vector*.

In [35]:
print(firstTransformation*messageIntVec)

[[  8.]
 [  1.]
 [ 14.]
 [ 19.]
 [ 19.]
 [792.]]


The nice thing about our application of linear algebra is that we're interested in transformations between integers *modulo 26*. When reflecting upon how the rest of our mathematics is going to work (have faith for the moment! :) ) we're okay recording results modulo 26 after each operation if we like, e.g.:

In [38]:
messageIntVec2 = firstTransformation*messageIntVec % 26
print(messageIntVec2)

[[ 8.]
 [ 1.]
 [14.]
 [19.]
 [19.]
 [12.]]


Taking results modulo 26 also allows us to interpret this result with a transformed *string* representation:

```
 8 -> i
 1 -> b
14 -> o
19 -> t
19 -> t
12 -> m
```

So now our string representation reads 'ibottm'. We can continue similarly - from this result - to encode each of the other letters of our message in turn.

## Forward transformation - encoding *all* letters
---
What we'll do now is generalize the pattern we used to construct the first transformation of our message's integer representation in such a way that we can transform *all* the letters of the message.

As we proceed, we'll see how we can multiply individual transformation matrices together into a *single* encryption matrix that transforms all the letters at once (and is equivalent to transforming each individual integer representation in succession).